# Derivation of the double pendulum
Consider the system, depicted below

![Sketch of the double pendulum system](system-sketch.svg)

The objective of this notebook is to derive the equations of motion of the system using [Lagrange's equations](https://en.wikipedia.org/wiki/Lagrangian_mechanics).  

In [8]:
using Pkg; Pkg.activate(".")
using SymPy
using LinearAlgebra

  Activating project at `C:\Users\fitzgeraldt\Documents\doublependulum\derivation`


In [15]:
dot(x,y) = sum( x .* y )
m1, m2, l1, l2, lp, g, Ig1, Ig2 = symbols("m_1, m_2, l_1, l_2, l_p, g, I_G1, I_G1", real=true, positive=true)
ϕ, t = symbols("phi, t", real=true)
θ1 = SymFunction("theta_1")(t)
θ2 = SymFunction("theta_2")(t)

ihat = [1, 0, 0]
jhat = [0, 1, 0]
khat = [0, 0, 1]
;

## Setup
The generalized coordinates are the absolute angles $q = [\theta_1,~ \theta_2]$.  The angle $\phi$ is fixed on the first body, and is the angle relative to the center of mass.

### The position vectors
All defined in the $\{\hat{i}\hat{j}\hat{k}\}$ system

In [18]:
r1 = l1*( sin(θ1)*ihat - cos(θ1)*jhat )
rp = lp*( sin(θ1+ϕ)*ihat - cos(θ1+ϕ)*jhat )
r2 = rp + l2*( sin(θ2)*ihat - cos(θ2)*jhat )
;

### The velocities

In [21]:
v1 = diff.( r1, t)

3-element Vector{Sym}:
 l_1*cos(theta_1(t))*Derivative(theta_1(t), t)
 l_1*sin(theta_1(t))*Derivative(theta_1(t), t)
                                             0

In [22]:
v2 = diff.( r2, t)

3-element Vector{Sym}:
 l_2*cos(theta_2(t))*Derivative(theta_2(t), t) + l_p*cos(phi + theta_1(t))*Derivative(theta_1(t), t)
 l_2*sin(theta_2(t))*Derivative(theta_2(t), t) + l_p*sin(phi + theta_1(t))*Derivative(theta_1(t), t)
                                                                                                   0

### The kinetic energy $T$

In [24]:
T1 = 1//2*m1*dot( v1, v1 ) + 1//2*Ig1*diff(θ1,t)^2 |> simplify

                                  2
/          2    \ /d             \ 
\I_G1 + l_1 *m_1/*|--(theta_1(t))| 
                  \dt            / 
-----------------------------------
                 2                 

In [25]:
T2 = 1//2*m2*dot( v2, v2 ) + 1//2*Ig2*diff(θ2,t)^2 |> simplify

                     2       /                     2                          
     /d             \        |   2 /d             \                           
I_G1*|--(theta_2(t))|    m_2*|l_2 *|--(theta_2(t))|  + 2*l_2*l_p*cos(phi + the
     \dt            /        \     \dt            /                           
---------------------- + -----------------------------------------------------
          2                                                                   

                                                                           2\
                      d              d                   2 /d             \ |
ta_1(t) - theta_2(t))*--(theta_1(t))*--(theta_2(t)) + l_p *|--(theta_1(t))| |
                      dt             dt                    \dt            / /
-----------------------------------------------------------------------------
           2                                                                 

In [26]:
T = T1 + T2 

                     2       /                     2                          
     /d             \        |   2 /d             \                           
I_G1*|--(theta_2(t))|    m_2*|l_2 *|--(theta_2(t))|  + 2*l_2*l_p*cos(phi + the
     \dt            /        \     \dt            /                           
---------------------- + -----------------------------------------------------
          2                                                                   

                                                                           2\ 
                      d              d                   2 /d             \ | 
ta_1(t) - theta_2(t))*--(theta_1(t))*--(theta_2(t)) + l_p *|--(theta_1(t))| | 
                      dt             dt                    \dt            / / 
----------------------------------------------------------------------------- 
           2                                                                  

                                    2
  /         

### The potential energy $V$

In [27]:
V1 = m1*g*dot(r1,jhat)

-g*l_1*m_1*cos(theta_1(t))

In [28]:
V2 = m2*g*dot(r2, jhat)

g*m_2*(-l_2*cos(theta_2(t)) - l_p*cos(phi + theta_1(t)))

In [29]:
V = V1 + V2 

-g*l_1*m_1*cos(theta_1(t)) + g*m_2*(-l_2*cos(theta_2(t)) - l_p*cos(phi + theta
_1(t)))

## Lagrange's equations

First, we build the Lagrangian $L = T-V$, and then use it in
$$ \frac{d}{dt}\left( \frac{\partial L}{\partial \dot{q}_k} \right) - \frac{\partial L}{\partial q_k} = 0, \quad k = 1,\,2$$



In [31]:
L = T-V |> simplify

                     2                                                        
     /d             \                                                         
I_G1*|--(theta_2(t))|                                                         
     \dt            /                                                         
---------------------- + g*l_1*m_1*cos(theta_1(t)) + g*m_2*(l_2*cos(theta_2(t)
          2                                                                   

                                     /                     2                  
                                     |   2 /d             \                   
                                 m_2*|l_2 *|--(theta_2(t))|  + 2*l_2*l_p*cos(p
                                     \     \dt            /                   
) + l_p*cos(phi + theta_1(t))) + ---------------------------------------------
                                                                              

                                                  

In [34]:
lag = [ diff(L, diff(q,t), t ) - diff( L, q ) |> simplify for q in [θ1, θ2] ]

2-element Vector{Sym}:
 I_G1*Derivative(theta_1(t), (t, 2)) + g*l_1*m_1*sin(theta_1(t)) + g*l_p*m_2*sin(phi + theta_1(t)) + l_1^2*m_1*Derivative(theta_1(t), (t, 2)) + l_2*l_p*m_2*sin(phi + theta_1(t) - theta_2(t))*Derivative(theta_2(t), t)^2 + l_2*l_p*m_2*cos(phi + theta_1(t) - theta_2(t))*Derivative(theta_2(t), (t, 2)) + l_p^2*m_2*Derivative(theta_1(t), (t, 2))
                                                                              I_G1*Derivative(theta_2(t), (t, 2)) + g*l_2*m_2*sin(theta_2(t)) + l_2^2*m_2*Derivative(theta_2(t), (t, 2)) - l_2*l_p*m_2*sin(phi + theta_1(t) - theta_2(t))*Derivative(theta_1(t), t)^2 + l_2*l_p*m_2*cos(phi + theta_1(t) - theta_2(t))*Derivative(theta_1(t), (t, 2))

## First order form
Let's work to rearrange this pair of coupled 2nd order equations to state-space (first order form) so we can use standard initial value ordinary differential equation solvers.
Let's pick the states
$$ z = [ \theta_1,\, \theta_2,\, \dot{\theta}_1,\, \dot{\theta}_2]^T $$


In [40]:
sol = solve( lag, [diff(θ1,t,t), diff(θ2, t, t)])

Dict{Any, Any} with 2 entries:
  Derivative(theta_2(t), (t, 2)) => -I_G1*g*l_2*m_2*sin(theta_2(t))/(I_G1^2 + I…
  Derivative(theta_1(t), (t, 2)) => -I_G1*g*l_1*m_1*sin(theta_1(t))/(I_G1^2 + I…

In [ ]:
# TODO: complete and output matlab/octave code